In [13]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

def load_data(data_folder, test_size=0.2, random_state=42):
    """ Load MyoWare EMG feature data and split into training/testing sets. """
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("Feature or label file not found!")

    X = np.load(feature_path)  # Shape: (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # Shape: (num_batches,)

    print(f"Loaded features: {X.shape}")
    print(f"Loaded labels: {y.shape}")

    # Flatten features: Reshape (num_batches, num_windows, 15, num_channels) → (num_samples, num_features)
    num_batches, num_windows, num_rows, num_channels = X.shape
    X = X.reshape(num_batches, -1)  # Flatten everything except batches

    num_classes = len(np.unique(y))
    test_size = max(num_classes, int(len(y) * 0.2))  # At least one sample per class

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )


    print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")
    return X_train, X_test, y_train, y_test


In [14]:
# Load data
data_folder = "windowed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)


Loaded features: (45, 59, 15, 1)
Loaded labels: (45,)
Training set: (36, 885), Testing set: (9, 885)


In [15]:
import numpy as np

# Check if any NaN exists in the dataset
print(f"NaNs in X_train: {np.isnan(X_train).sum()} / {X_train.size}")
print(f"NaNs in X_test: {np.isnan(X_test).sum()} / {X_test.size}")

mask_train = ~np.isnan(X_train).any(axis=1)
mask_test = ~np.isnan(X_test).any(axis=1)

X_train, y_train = X_train[mask_train], y_train[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]

print()
# Check if any NaN exists in the dataset
print(f"NaNs in X_train: {np.isnan(X_train).sum()} / {X_train.size}")
print(f"NaNs in X_test: {np.isnan(X_test).sum()} / {X_test.size}")


NaNs in X_train: 10 / 31860
NaNs in X_test: 0 / 7965

NaNs in X_train: 0 / 29205
NaNs in X_test: 0 / 7965


In [16]:
# Train SVM Classifier
svm_clf = SVC(kernel='rbf', C=1, gamma='scale')
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)

# Evaluate models
print("\n=== SVM Results ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(classification_report(y_test, y_pred_svm))


=== SVM Results ===
Accuracy: 0.4444
              precision    recall  f1-score   support

           1       0.33      0.33      0.33         3
           2       0.50      1.00      0.67         3
           3       0.00      0.00      0.00         3

    accuracy                           0.44         9
   macro avg       0.28      0.44      0.33         9
weighted avg       0.28      0.44      0.33         9



c:\Users\theof\miniconda3\envs\grad_env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\theof\miniconda3\envs\grad_env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\theof\miniconda3\envs\grad_env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [18]:
# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)

print("\n=== KNN Results ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print(classification_report(y_test, y_pred_knn))


=== KNN Results ===
Accuracy: 0.6667
              precision    recall  f1-score   support

           1       0.50      0.67      0.57         3
           2       0.75      1.00      0.86         3
           3       1.00      0.33      0.50         3

    accuracy                           0.67         9
   macro avg       0.75      0.67      0.64         9
weighted avg       0.75      0.67      0.64         9

